In [ ]:
import numpy as np
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity

In [ ]:
#Determines if SMILES is valid or not
def is_valid(smiles):
    
    mol = Chem.MolFromSmiles(smiles)

    #Returns True if SMILES is valid, returns False if SMILES is invalid
    return smiles != '' and mol is not None and mol.GetNumAtoms() > 0

In [ ]:
#Determines LogP
def logP(smiles):
    
    return(Descriptors.MolLogP(Chem.MolFromSmiles(smiles)))

#Determines molecular weight
def molWt(smiles):
    
    return(Descriptors.MolWt(Chem.MolFromSmiles(smiles)))

#Determines number hydrogen bond acceptors
def numAcc(smiles):
    
    return(Descriptors.NumHAcceptors(Chem.MolFromSmiles(smiles)))

#Determines number hydrogen bond donors
def numDon(smiles):
    
    return(Descriptors.NumHDonors(Chem.MolFromSmiles(smiles)))

#Determines topological polar surface area
def polSur(smiles):
    
    return(Descriptors.TPSA(Chem.MolFromSmiles(smiles)))

#Determines number of rotatable bonds
def rolBon(smiles):
    return(Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smiles)))

In [ ]:
#Number of characters generated
num_characters = 0

#Number of molecules generated
num_smiles = 0

#Number of unique SMILES
num_unq_mols = 0

#Number of unique SMILES that aren't in the training data
num_mols = 0

#Number of valid molecules that aren't in the training data generated
num_valid = 0

#List of smiles in file, to make sure smiles are unique
smileslist = []

In [ ]:
#Test how many molecules are valid (without considering uniqueness, novelty)
total = 0
number_valid = 0

#Read in data file line by line
for line in open("gen.txt", "r"):
    total += 1
    
    #Ensure smiles are valid
    if(is_valid(line) == True):
                           
        #Increment number of valid molecules generated
        number_valid += 1

print("Percent valid molecules: " + str(number_valid / total * 100))

In [ ]:
#Training data
training_data = list(open("smiles.txt", "r"))

#File with unique generated SMILES that aren't in the training data
generatedmols = open("fgenmols.txt", "w")

#Read in data file line by line
for line in open("gen.txt", "r"):
    
    #Ensure molecules are unique
    if line not in smileslist:
        
        smileslist.append(line)
        
        num_unq_mols += 1

        #Ensure smiles aren't in training data
        if line not in training_data:  

            #Remove \n character, remove G character
            smiles = line.replace("\n", "").replace("G", "")
            
            #Increment number of molecules generated
            num_mols += 1

            #Ensure smiles are valid
            if(is_valid(smiles) == True):
            
                #Copy over SMILES satisfying requirements
                generatedmols.write(smiles + "\n")
                
                #Increment number of valid molecules generated
                num_valid += 1
                
    #Increment total number of SMILES generated
    num_smiles += 1
    
    #Add length of line to total number of characters
    num_characters += len(line)

In [ ]:
print("Number of characters generated: " + str(num_characters))
print("Number of molecules generated: " + str(num_smiles))
print("Number of unique molecules generated: " + str(num_unq_mols))
print("Number of novel and unique molecules generated: " + str(num_mols))
print("Number of novel, unique, and valid molecules generated: " + str(num_valid))

In [ ]:
#List of Morgan fingerprints of molecules
fingerprints = []

#Read in data file line by line
for line in open("genmols.txt", "r"):
    line = line.replace("G","")
    #Convert SMILES string to Morgan fingerprint
    mol = Chem.MolFromSmiles(line.replace("\n", ""))
    fingerprint = AllChem.GetMorganFingerprint(mol, 2)
    
    #Add to list of fingerprints
    fingerprints.append(fingerprint)

In [ ]:
import random

#Total Tanimoto Distance
tanimoto = 0

nummols = 1000
randfings = random.sample(fingerprints, nummols)

#Calculate Tanimoto Distance between each pair of fingerprints
for fpt1 in randfings:
    for fpt2 in randfings:
        
        if fpt1 != fpt2:
            
            #Calculate Tanimoto Similarity
            tan = TanimotoSimilarity(fpt1, fpt2)
            tanimoto += tan

#Average Tanimoto Similarity
avg_tanimoto = (1 / (nummols * (nummols - 1))) * tanimoto
print("Average Tanimoto Similarity: {:0.4f}".format(avg_tanimoto))

In [ ]:
#Cleaned data file
fgenmols = open("fgenmols.txt", "w")

#Combine best molecules with those generated in this iteration (maintains the best molecules throughout for faster convergence)
filenames = ["genmols.txt", "finestmols.txt"]
with open("allmols.txt", "w") as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
                
fgenmols.close()

In [ ]:
#Array of molecular properties for generated molecules
molProps = np.empty((0,5))

#Read in data file line by line
for molecule in open("genmols.txt", "r"):
    try:
        #Array of properties 
        props = np.reshape(np.array([logP(molecule), molWt(molecule), numAcc(molecule), numDon(molecule), rolBon(molecule)]), (1, 5))
        
        #Append properties
        molProps = np.append(molProps, props, axis=0)
        
        #Write molecules to final, cleaned dataset
        fgenmols.write(molecule)
        
    except: 
        #Occasionally RDKit bugs don't allow for analyzing the molecule; in these cases, do not include the molecule in the final dataset
        continue
        
fgenmols.close()

In [ ]:
#Array of number of molecules each molecule is dominated by
dom = np.zeros((np.shape(molProps)[0]))

#Analyze each molecule's properties as they compare to others
for i in range(np.shape(molProps)[0]):
    
    for j in range(np.shape(molProps)[0]):
        
        #Compare each property between the molecules
        if all((molProps[j, k] <= molProps[i, k]) for k in range(np.shape(molProps)[1])) and (np.array_equal(molProps[i,:], molProps[j,:]) == False):
                            
            #if molecule j is better than or equal to molecule i in every property, but not equal to i, than j dominates i
            dom[i] += 1

In [ ]:
#Fraction of molecules to be selected
top = 0.5

#Max number of molecules to be selected
maxmol = 10000

#Select the best molecules based on fraction / max number 
if (int(top * (np.shape(molProps)[0])) < maxmol):
    #Select the best of the molecules
    finestmols = np.argpartition(dom, int(top * (np.shape(molProps)[0])))
else:
    #Select the best of the molecules
    finestmols = np.argpartition(dom, maxmol)
    
finestmols = finestmols[:int(top * (np.shape(molProps)[0]))]

In [ ]:
#Save best molecules 
transferdata = open("finestmols.txt", "w")

i = 0
for line in open("fgenmols.txt", "r"):
    if i in finestmols:
        #Append start token
        line = line.rjust(len(line)+1, "G")
        
        #Write to file
        transferdata.write(line)
    
    i += 1 

transferdata.close()

In [ ]:
#Read in original training data file to get character list
data = open("smiles.txt", "r").read()

#Create a list of the unique characters in the dataset
chars = list(set(data))

In [ ]:
from numpy import argmax
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Create array from characters in the dataset
values = array(chars)
valueslist = values.tolist()
print("Array of unique characters:")
print(values)

#Create unique, numerical labels for each character between 0 and n-1, where n is the number of unique characters
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Array of labels for each character:")
print(integer_encoded)

#Encode characters into a one-hot encoding, resulting in an array of size [num unique chars, num unique chars]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print("Array of one-hot encoded characters:")
print(onehot_encoded)
print("Size of array of one-hot encoded characters: " + str(onehot_encoded.shape))

In [ ]:
#Read in data file of best molecules
data = open("finestmols.txt", "r").read()
#Create a list of the dataset, along with all the characters to ensure they are all represented
datalist = list(data)
datalist.extend(valueslist)
#Create an array of the dataset
dataarray = array(datalist)
#Fit one-hot encoding to dataarray
dataarray = dataarray.reshape(len(dataarray), 1)
#Fit encoder, remove all characters at the end leaving just the molecules
ohefinemols = onehot_encoder.fit_transform(dataarray).astype(int)
print("Size of one-hot encoded array of data: " + str(ohefinemols.shape))
print("One-hot encoded array of data:")
print(ohefinemols)

In [ ]:
#Save ohefinemols as a (compressed) file
np.savez_compressed("ohefinemols.npz", ohefinemols)

In [ ]:
#Create integer transfer data
intfinemols = [np.where(r==1)[0][0] for r in ohefinemols]

In [ ]:
#Save intfinemols as a (compressed) file
np.savez_compressed("intfinemols.npz", intfinemols)